# TabletGaze Cross Data Generalization

This notebook will guide you through the process of recreating the generalization with TabletGaze.
First, you should obtain the dataset and extract its contents to a folder named `TabletGazeDataset`.

Next, run each following code block one by one.

Note:
Due to the amount of time to train the algorithm, we weren't able to retrain the model on the TabletGaze dataset. Instead, we took the pre-built model (that was provided with the source code) and ran validation on it. Through this, we obtained a RMSE of 2.12 (representing centimeters).

## Boilerplate
Run this code to register all boilerplate code for future use.

In [1]:
'''
Parses frames out of a video file for the use of testing the images.
'''

import numpy as np
import cv2
import av
import mediapipe as mp
import math
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from threading import local
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import h5py
import torch.utils.data as data
import scipy.io as sio
from PIL import Image
import os
import os.path
import torchvision.transforms as transforms

# This code is converted from https://github.com/CSAILVision/GazeCapture/blob/master/code/faceGridFromFaceRect.m

# Given face detection data, generate face grid data.
#
# Input Parameters:
# - frameW/H: The frame in which the detections exist
# - gridW/H: The size of the grid (typically same aspect ratio as the
#     frame, but much smaller)
# - labelFaceX/Y/W/H: The face detection (x and y are 0-based images
#     coordinates)
# - parameterized: Whether to actually output the grid or just the
#     [x y w h] of the 1s square within the gridW x gridH grid.
# https://drive.google.com/drive/folders/1ZcYb4eH2jPndS5nkqQFcLHdGNM9dTF5C?usp=sharing
# https://drive.google.com/file/d/gpip1UdhuJ_bulreFyGa8CziK4tdwXeHC2PIh/view?usp=sharing

def generate_centered_face_grid(gridW, gridH):
    labelFaceGrid = np.zeros(gridW * gridH)
    grid = np.zeros((gridH, gridW))

    xLo = 7
    yLo = 7

    xHi = 18
    yHi = 18

    faceLocation = np.ones((yHi - yLo, xHi - xLo))
    grid[yLo:yHi, xLo:xHi] = faceLocation

    # Flatten the grid.
    grid = np.transpose(grid)
    labelFaceGrid = grid.flatten()

    return labelFaceGrid

def faceGridFromFaceRect(frameW, frameH, gridW, gridH, labelFaceX, labelFaceY, labelFaceW, labelFaceH, parameterized):

    scaleX = gridW / frameW
    scaleY = gridH / frameH

    if parameterized:
      labelFaceGrid = np.zeros(4)
    else:
      labelFaceGrid = np.zeros(gridW * gridH)

    grid = np.zeros((gridH, gridW))

    # Use one-based images coordinates.
    xLo = round(labelFaceX * scaleX)
    yLo = round(labelFaceY * scaleY)
    w = round(labelFaceW * scaleX)
    h = round(labelFaceH * scaleY)

    if parameterized:
        labelFaceGrid = [xLo, yLo, w, h]
    else:
        xHi = xLo + w
        yHi = yLo + h

        # Clamp the values in the range.
        xLo = int(min(gridW, max(0, xLo)))
        xHi = int(min(gridW, max(0, xHi)))
        yLo = int(min(gridH, max(0, yLo)))
        yHi = int(min(gridH, max(0, yHi)))

        faceLocation = np.ones((yHi - yLo, xHi - xLo))
        grid[yLo:yHi, xLo:xHi] = faceLocation

        # Flatten the grid.
        grid = np.transpose(grid)
        labelFaceGrid = grid.flatten()

    return labelFaceGrid

def get_face_grid(face, frameW, frameH, gridSize):
    faceX,faceY,faceW,faceH = face

    return faceGridFromFaceRect(frameW, frameH, gridSize, gridSize, faceX, faceY, faceW, faceH, True)


def crop_to_bounds(img, bounds):
    [x, y, w, h] = bounds
    cropped = img[y:y + h, x:x + w]
    return cropped


gridSize = 25

def get_frames(video_file, stream=None):
    """
    Parses all frames out of the given video file and returns an array of PIL images.
    """

    container = av.open(video_file)
    video = container.streams.video[0]

    to_return = []
    for idx, frame in enumerate(container.decode(video)):
        image = cv2.cvtColor(frame.to_rgb().to_ndarray(), cv2.COLOR_RGB2BGR)
        frame_time = float(frame.pts * video.time_base)
        if stream is None or not callable(stream):
            to_return.append([image, frame_time])
        else:
            stream(image, frame_time, idx)
    container.close()

    return to_return

detector_storage = local()

def detect_features(np_img):
    if not hasattr(detector_storage, "detector"):
        detector_storage.base_options = python.BaseOptions(model_asset_path='detector.tflite')
        detector_storage.options = vision.FaceDetectorOptions(base_options=detector_storage.base_options)
        detector_storage.detector = vision.FaceDetector.create_from_options(detector_storage.options)
    
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np_img)
    dresult = detector_storage.detector.detect(mp_image)
    if len(dresult.detections) == 0:
        return [np_img, [], []]

    im_width = np_img.shape[1]
    im_height = np_img.shape[0]
    detection = dresult.detections[0]
    right_eye = detection.keypoints[0]
    left_eye = detection.keypoints[1]

    face_bbx = detection.bounding_box
    built_face_bbx = [face_bbx.origin_x, face_bbx.origin_y, face_bbx.width, face_bbx.height]
    face_size = face_bbx.width

    right_eye_px = [math.floor(right_eye.x * im_width), math.floor(right_eye.y * im_height)]
    left_eye_px = [math.floor(left_eye.x * im_width), math.floor(left_eye.y * im_height)]

    eye_ratio = math.ceil(face_size / 8)
    right_eye_bbx = [right_eye_px[0] - eye_ratio, right_eye_px[1] - eye_ratio, eye_ratio * 2, eye_ratio * 2]
    left_eye_bbx = [left_eye_px[0] - eye_ratio, left_eye_px[1] - eye_ratio, eye_ratio * 2, eye_ratio * 2]

    return np_img, [built_face_bbx], [[right_eye_bbx, left_eye_bbx],
                                      get_face_grid(built_face_bbx, im_width, im_height, 25)]

'''
Pytorch model for the iTracker.

Author: Petr Kellnhofer ( pkel_lnho (at) gmai_l.com // remove underscores and spaces), 2018. 

Website: http://gazecapture.csail.mit.edu/

Cite:

Eye Tracking for Everyone
K.Krafka*, A. Khosla*, P. Kellnhofer, H. Kannan, S. Bhandarkar, W. Matusik and A. Torralba
IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016

@inproceedings{cvpr2016_gazecapture,
Author = {Kyle Krafka and Aditya Khosla and Petr Kellnhofer and Harini Kannan and Suchendra Bhandarkar and Wojciech Matusik and Antonio Torralba},
Title = {Eye Tracking for Everyone},
Year = {2016},
Booktitle = {IEEE Conference on Computer Vision and Pattern Recognition (CVPR)}
}

'''


class ItrackerImageModel(nn.Module):
    # Used for both eyes (with shared weights) and the face (with unqiue weights)
    def __init__(self):
        super(ItrackerImageModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.CrossMapLRN2d(size=5, alpha=0.0001, beta=0.75, k=1.0),
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2, groups=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.CrossMapLRN2d(size=5, alpha=0.0001, beta=0.75, k=1.0),
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 64, kernel_size=1, stride=1, padding=0),
            nn.ReLU(inplace=True),
            
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return x

class FaceImageModel(nn.Module):
    
    def __init__(self):
        super(FaceImageModel, self).__init__()
        self.conv = ItrackerImageModel()
        self.fc = nn.Sequential(
            nn.Linear(12*12*64, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x

class FaceGridModel(nn.Module):
    # Model for the face grid pathway
    def __init__(self, gridSize = 25):
        super(FaceGridModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(gridSize * gridSize, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x



class ITrackerModel(nn.Module):


    def __init__(self):
        super(ITrackerModel, self).__init__()
        self.eyeModel = ItrackerImageModel()
        self.faceModel = FaceImageModel()
        self.gridModel = FaceGridModel()
        # Joining both eyes
        self.eyesFC = nn.Sequential(
            nn.Linear(2*12*12*64, 128),
            nn.ReLU(inplace=True),
            )
        # Joining everything
        self.fc = nn.Sequential(
            nn.Linear(128+64+128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 2),
            )

    def forward(self, faces, eyesLeft, eyesRight, faceGrids):
        # Eye nets
        xEyeL = self.eyeModel(eyesLeft)
        xEyeR = self.eyeModel(eyesRight)
        # Cat and FC
        xEyes = torch.cat((xEyeL, xEyeR), 1)
        xEyes = self.eyesFC(xEyes)

        # Face net
        xFace = self.faceModel(faces)
        xGrid = self.gridModel(faceGrids)

        # Cat all
        x = torch.cat((xEyes, xFace, xGrid), 1)
        x = self.fc(x)
        
        return x

'''
Data loader for the iTracker.
Use prepareDataset.py to convert the dataset from http://gazecapture.csail.mit.edu/ to proper format.

Author: Petr Kellnhofer ( pkel_lnho (at) gmai_l.com // remove underscores and spaces), 2018. 

Website: http://gazecapture.csail.mit.edu/

Cite:

Eye Tracking for Everyone
K.Krafka*, A. Khosla*, P. Kellnhofer, H. Kannan, S. Bhandarkar, W. Matusik and A. Torralba
IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016

@inproceedings{cvpr2016_gazecapture,
Author = {Kyle Krafka and Aditya Khosla and Petr Kellnhofer and Harini Kannan and Suchendra Bhandarkar and Wojciech Matusik and Antonio Torralba},
Title = {Eye Tracking for Everyone},
Year = {2016},
Booktitle = {IEEE Conference on Computer Vision and Pattern Recognition (CVPR)}
}

'''

MEAN_PATH = './'

def clamp_eyes_to_frame(box, width, height):
    if box[0] > 60000:
        box[0] = 0
    if box[1] > 60000:
        box[1] = 0
    
    if box[0] > 1000:
        box[0] = width
    if box[1] > 1000:
        box[1] = height

def loadMetadata(filename, silent = False, struct_as_record = False):
    try:
        # http://stackoverflow.com/questions/6273634/access-array-contents-from-a-mat-file-loaded-using-scipy-io-loadmat-python
        if not silent:
            print('\tReading metadata from %s...' % filename)
        metadata = sio.loadmat(filename, squeeze_me=True, struct_as_record=struct_as_record)
    except:
        print('\tFailed to read the meta file "%s"!' % filename)
        return None
    return metadata

class SubtractMean(object):
    """Normalize an tensor images with mean.
    """

    def __init__(self, meanImg):
        self.meanImg = transforms.ToTensor()(meanImg / 255)

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor images of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized images.
        """       
        return tensor.sub(self.meanImg)

class TabletGazePreprocessData(data.Dataset):
    def __init__(self, dataPath, split='all', imSize=(224, 224), gridSize=(25, 25)):
        """
        The tablet gaze dataset has extra metadata for each video...

        The first, stored in gazePts.mat, stores the order and location of the dot appearance for ALL trials.
        The second, stored in startTime.mat, stores the time (in seconds)? that the trial begins. (Discard all frames prior to this timestamp)

        The trial was conducted with 35 dot positions. After the start time, every 3 seconds the dot changes positions.
        """
        self.data_path = dataPath
        self.imSize = imSize
        self.gridSize = gridSize

        self.gaze_pts = loadMetadata(os.path.join(dataPath, 'gazePts.mat'), struct_as_record=True)['gazePts'].tolist()
        self.gaze_points_x = self.gaze_pts[2]
        self.gaze_points_y = self.gaze_pts[3]
        self.startTime = loadMetadata(os.path.join(dataPath, 'startTime.mat'), struct_as_record=True)['startTime'].tolist()

        self.subjects = 51
        self.screenWidthCM = 22.62
        self.screenHeightCM = 14.14

        self.screenCenterXCM = self.screenWidthCM / 2
        self.screenCenterYCM = self.screenHeightCM / 2

        if split == 'train':
            subject_split = range(0, math.ceil(self.subjects * 0.8)) # Use 80% of the set for training
        elif split == 'test':
            subject_split = range(math.ceil(self.subjects * 0.8), self.subjects) # Use 20% of the set for testing
        else:
            subject_split = range(0, self.subjects)

        self.indices = []
        for subject in subject_split:
            for trial in range(0, 4):
                for pose in range(0, 4):
                    self.indices.append([subject + 1, trial + 1, pose + 1])


    def makeGrid(self, params):
        gridLen = self.gridSize[0] * self.gridSize[1]
        grid = np.zeros([gridLen, ], np.float32)

        indsY = np.array([i // self.gridSize[0] for i in range(gridLen)])
        indsX = np.array([i % self.gridSize[0] for i in range(gridLen)])
        condX = np.logical_and(indsX >= params[0], indsX < params[0] + params[2])
        condY = np.logical_and(indsY >= params[1], indsY < params[1] + params[3])
        cond = np.logical_and(condX, condY)

        grid[cond] = 1
        return grid

    def get_dot_index(self, subject, trial, pose, current_time):
        subject_set = self.startTime[subject]
        time_start = subject_set[trial][pose]
        # If the trial hasn't started yet, don't process it!
        if current_time < time_start:
            return -1

        # There is a new dot every 3 seconds.
        dot_index = math.floor((current_time - time_start) / 3)
        return dot_index

    def __getitem__(self, index):
        """
        Gets a batch of frames from the given video index.
        """
        subject, trial, pose = self.indices[index]
        data_points = []
        video_name = f'{subject}/{subject}_{trial}_{pose}.mp4'
        path_to_video = os.path.join(self.data_path, video_name)
        print('Loading frames for %s' % path_to_video)
        num = 0
        frame_count = 0

        if not os.path.exists(os.path.join(self.data_path, f"{subject}_{trial}_{pose}_image")):
            os.makedirs(os.path.join(self.data_path, f"{subject}_{trial}_{pose}_image"))
        def parse_frame(frame, frame_time, frame_idx):
            nonlocal num, frame_count
            num += 1
            if num % 1000 == 0:
                print('Loading frame %s for video file %s' % (num, path_to_video))

            # Get the dot index and check if the trial hasn't started or has already finished.
            dot_index = self.get_dot_index(subject - 1, trial - 1, pose - 1, frame_time)
            if dot_index < 0 or dot_index >= 35:
                return

            features = detect_features(frame)

            faces = features[1]
            # Skip the face if it wasn't detected!
            if len(faces) == 0:
                return

            imFace = crop_to_bounds(frame, faces[0])
            right_eye, left_eye = features[2][0]

            face_x = faces[0][0]
            face_y = faces[0][1]

            # Offset eyes by this amount
            right_eye[0] -= face_x
            right_eye[1] -= face_y

            left_eye[0] -= face_x
            left_eye[1] -= face_y

            faceGrid = self.makeGrid(features[2][1])
            cv2.imwrite(os.path.join(self.data_path, f"{subject}_{trial}_{pose}_image/{frame_count}.jpg"), imFace)

            # to tensor
            data_points.append(
                [np.array(faces[0]), np.array(left_eye), np.array(right_eye), faceGrid, frame_time, num - 1])
            frame_count += 1

        get_frames(path_to_video, parse_frame)
        return [data_points, subject, trial, pose]


    def __len__(self):
        return len(self.indices)

class TabletGazePostprocessData(data.Dataset):
    def __init__(self, dataPath, preprocess_path='preprocessed.h5', split='all', imSize=(224, 224), gridSize=(25, 25)):
        """
        The tablet gaze dataset has extra metadata for each video...

        The first, stored in gazePts.mat, stores the order and location of the dot appearance for ALL trials.
        The second, stored in startTime.mat, stores the time (in seconds)? that the trial begins. (Discard all frames prior to this timestamp)

        The trial was conducted with 35 dot positions. After the start time, every 3 seconds the dot changes positions.
        """
        self.data_path = dataPath
        self.imSize = imSize
        self.gridSize = gridSize

        self.gaze_pts = loadMetadata(os.path.join(dataPath, 'gazePts.mat'), struct_as_record=True)['gazePts'].tolist()
        self.gaze_points_x = self.gaze_pts[2]
        self.gaze_points_y = self.gaze_pts[3]
        self.startTime = loadMetadata(os.path.join(dataPath, 'startTime.mat'), struct_as_record=True)['startTime'].tolist()

        self.faceMean = loadMetadata(os.path.join(MEAN_PATH, 'mean_face_224.mat'))['image_mean']
        self.eyeLeftMean = loadMetadata(os.path.join(MEAN_PATH, 'mean_left_224.mat'))['image_mean']
        self.eyeRightMean = loadMetadata(os.path.join(MEAN_PATH, 'mean_right_224.mat'))['image_mean']

        self.transformFace = transforms.Compose([
            transforms.Resize(self.imSize),
            transforms.ToTensor(),
            SubtractMean(meanImg=self.faceMean),
        ])
        self.transformEyeL = transforms.Compose([
            transforms.Resize(self.imSize),
            transforms.ToTensor(),
            SubtractMean(meanImg=self.eyeLeftMean),
        ])
        self.transformEyeR = transforms.Compose([
            transforms.Resize(self.imSize),
            transforms.ToTensor(),
            SubtractMean(meanImg=self.eyeRightMean),
        ])

        self.subjects = 5 #51
        self.preprocess_path = preprocess_path
        self.screenWidthCM = 22.62
        self.screenHeightCM = 14.14

        self.screenCenterXCM = self.screenWidthCM / 2
        self.screenCenterYCM = self.screenHeightCM / 2

        if split == 'train':
            subject_split = range(0, math.ceil(self.subjects * 0.8)) # Use 80% of the set for training
        elif split == 'test':
            subject_split = range(math.ceil(self.subjects * 0.8), self.subjects) # Use 20% of the set for testing
        else:
            subject_split = range(0, self.subjects)

        self.indices = []
        self.h5file = h5py.File(self.preprocess_path, 'r')
        for subject in subject_split:
            for trial in range(0, 1):
                for pose in range(0, 4):
                    for file in os.listdir(os.path.join(self.data_path, f"{subject + 1}_{trial + 1}_{pose + 1}_image")):
                        if ".jpg" not in file:
                            continue

                        self.indices.append([subject + 1, trial + 1, pose + 1, int(file.replace(".jpg", ""))])


    def makeGrid(self, params):
        gridLen = self.gridSize[0] * self.gridSize[1]
        grid = np.zeros([gridLen, ], np.float32)

        indsY = np.array([i // self.gridSize[0] for i in range(gridLen)])
        indsX = np.array([i % self.gridSize[0] for i in range(gridLen)])
        condX = np.logical_and(indsX >= params[0], indsX < params[0] + params[2])
        condY = np.logical_and(indsY >= params[1], indsY < params[1] + params[3])
        cond = np.logical_and(condX, condY)

        grid[cond] = 1
        return grid

    def get_dot_index(self, subject, trial, pose, current_time):
        subject_set = self.startTime[subject]
        time_start = subject_set[trial][pose]
        # If the trial hasn't started yet, don't process it!
        if current_time < time_start:
            return -1

        # There is a new dot every 3 seconds.
        dot_index = math.floor((current_time - time_start) / 3)
        return dot_index


    def __getitem__(self, index):
        """
        Gets a batch of frames from the given video index.
        """
        subject, trial, pose, frame_num = self.indices[index]
        file_name = f"{subject}_{trial}_{pose}_image/{frame_num}.jpg"

        eyes_l = self.h5file.get(f'{subject}/{trial}_{pose}/eyes_l')[()]
        eyes_r = self.h5file.get(f'{subject}/{trial}_{pose}/eyes_r')[()]
        metadata_loaded = self.h5file.get(f'{subject}/{trial}_{pose}/metadata')[()]
        frametime_loaded = self.h5file.get(f'{subject}/{trial}_{pose}/frame_times')[()]
        frame_indices = self.h5file.get(f'{subject}/{trial}_{pose}/frame_indices')[()]

        if frame_num >= len(frame_indices):
            return

        face_grid = metadata_loaded[frame_num]
        frame_time = frametime_loaded[frame_num]
        leye = eyes_l[frame_num]
        reye = eyes_r[frame_num]

        # Get the dot index and check if the trial hasn't started or has already finished.
        dot_index = self.get_dot_index(subject - 1, trial - 1, pose - 1, frame_time)
        if dot_index < 0 or dot_index >= 35:
            return

        topleft_dot_x_cm = self.gaze_points_x[dot_index]
        topleft_dot_y_cm = self.gaze_points_y[dot_index]

        # We need to transform the dot locations to be based from the center of the screen, which is what iTracker uses.
        dot_x_cm = topleft_dot_x_cm - self.screenCenterXCM
        dot_y_cm = -topleft_dot_y_cm
        frame = cv2.imread(os.path.join(self.data_path, file_name))

        gaze = np.array([dot_x_cm, dot_y_cm], np.float32)
        clamp_eyes_to_frame(leye, frame.shape[1], frame.shape[0])
        clamp_eyes_to_frame(reye, frame.shape[1], frame.shape[0])

        imFace = frame
        imEyeR = crop_to_bounds(frame, reye)
        imEyeL = crop_to_bounds(frame, leye)

        imFace = Image.fromarray(cv2.cvtColor(imFace, cv2.COLOR_BGR2RGB))
        imEyeR = Image.fromarray(cv2.cvtColor(imEyeR, cv2.COLOR_BGR2RGB))
        imEyeL = Image.fromarray(cv2.cvtColor(imEyeL, cv2.COLOR_BGR2RGB))

        imFace = self.transformFace(imFace)
        imEyeR = self.transformEyeR(imEyeR)
        imEyeL = self.transformEyeL(imEyeL)

        face_grid = torch.FloatTensor(face_grid)
        gaze = torch.FloatTensor(gaze)

        # print("Yes. We're still running.")
        return [imFace, imEyeL, imEyeR, face_grid, gaze]


    def __len__(self):
        return len(self.indices)

## Preprocessing
Run this code to preprocess and prepare the dataset.

In [2]:
import h5py
import numpy as np
import torch.nn.parallel
import torch.optim
import torch.utils.data
import tqdm


def preprocess_dataset(dataset_path, output_path='preprocessed.h5'):
    """
    Produces a .h5 file with annotated data.
    """
    print(os.getcwd())

    # Create the data loader and load all frames.
    loader = torch.utils.data.DataLoader(TabletGazePreprocessData(dataset_path), pin_memory=True, shuffle=False, num_workers=24)

    # if os.path.exists(output_path):
    #     os.remove(output_path)

    for video_data in tqdm.tqdm(loader):
        face = []
        eye_l = []
        eye_r = []
        metadata_record = []
        frame_times = []
        frame_indices = []
        frame_group = video_data[0]
        subject = video_data[1].item()
        trial = video_data[2].item()
        pose = video_data[3].item()
        for frame_data in frame_group:
            imFace, imEyeL, imEyeR, faceGrid, frame_time, index = frame_data
            face.append(imFace.numpy().copy()[0])
            eye_l.append(imEyeL.numpy().copy()[0])
            eye_r.append(imEyeR.numpy().copy()[0])
            frame_times.append(frame_time[0].item())
            metadata_record.append(faceGrid.numpy().copy()[0])
            frame_indices.append(index.numpy().copy()[0])

        face_encode = np.array(face).astype(np.uint16)
        eye_l_encode = np.array(eye_l).astype(np.uint16)
        eye_r_encode = np.array(eye_r).astype(np.uint16)
        metadata_encode = np.array(metadata_record).astype(np.float32)
        frame_indices_encode = np.array(frame_indices).astype(np.uint32)
        frame_times_encode = np.array(frame_times).astype(np.float32)
        with h5py.File(output_path, 'a') as output:
            output.create_dataset(f'{subject}/{trial}_{pose}/faces', data=face_encode)
            output.create_dataset(f'{subject}/{trial}_{pose}/eyes_l', data=eye_l_encode)
            output.create_dataset(f'{subject}/{trial}_{pose}/eyes_r', data=eye_r_encode)
            output.create_dataset(f'{subject}/{trial}_{pose}/metadata', data=metadata_encode)
            output.create_dataset(f'{subject}/{trial}_{pose}/frame_indices', data=frame_indices_encode)
            output.create_dataset(f'{subject}/{trial}_{pose}/frame_times', data=frame_times_encode)


if __name__ == '__main__':
    preprocess_dataset('TabletGazeDataset')


/NewRaidData/atb00010_cs460/GazeCapture7
	Reading metadata from TabletGazeDataset/gazePts.mat...
	Reading metadata from TabletGazeDataset/startTime.mat...


  0%|          | 0/816 [00:00<?, ?it/s]

Loading frames for TabletGazeDataset/1/1_1_2.mp4Loading frames for TabletGazeDataset/1/1_1_1.mp4Loading frames for TabletGazeDataset/1/1_1_3.mp4Loading frames for TabletGazeDataset/1/1_1_4.mp4Loading frames for TabletGazeDataset/1/1_2_1.mp4Loading frames for TabletGazeDataset/1/1_2_2.mp4Loading frames for TabletGazeDataset/1/1_2_4.mp4Loading frames for TabletGazeDataset/1/1_3_1.mp4

Loading frames for TabletGazeDataset/1/1_3_2.mp4Loading frames for TabletGazeDataset/1/1_3_3.mp4


Loading frames for TabletGazeDataset/1/1_3_4.mp4
Loading frames for TabletGazeDataset/1/1_4_1.mp4
Loading frames for TabletGazeDataset/1/1_4_2.mp4




Loading frames for TabletGazeDataset/1/1_4_3.mp4
Loading frames for TabletGazeDataset/1/1_4_4.mp4Loading frames for TabletGazeDataset/2/2_1_1.mp4
Loading frames for TabletGazeDataset/2/2_1_2.mp4

Loading frames for TabletGazeDataset/2/2_1_3.mp4
Loading frames for TabletGazeDataset/2/2_1_4.mp4
Loading frames for TabletGazeDataset/2/2_2_1.mp4Loading frames for Tab

I0000 00:00:1702223905.064839  889005 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.071634  888725 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.071959  889045 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.083600  888845 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.096350  888525 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.168699  888565 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.173947  888926 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1702223905.327162  890873 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.104.05), renderer: NVIDIA TITAN RTX/PCIe/SSE2
I0000 00:00:1702223905.327494  890872 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVID

KeyboardInterrupt: 

Loading frame 1000 for video file TabletGazeDataset/1/1_2_1.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_2_1.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_2_2.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_1_3.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_3_1.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_1_1.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_4_4.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_4_1.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_2_3.mp4
Loading frame 1000 for video file TabletGazeDataset/2/2_1_4.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_1_2.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_1_1.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_1_3.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_3_4.mp4
Loading frame 1000 for video file TabletGazeDataset/1/1_2_4.mp4
Loading frame 1000 for video file Tablet

## Validation
Run this code to reproduce the validation results.

In [ ]:
# Currently achieves 4.5059 (2.12) on the validation set.
# Currently achieves 14.8343 (3.85) on MPIIGaze validation.
# Currently achieves 8.9463 (2.99) on true MPIIGaze cross generalization

import shutil, os, time, argparse

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data

'''
Train/test code for iTracker.

Author: Petr Kellnhofer ( pkel_lnho (at) gmai_l.com // remove underscores and spaces), 2018. 

Website: http://gazecapture.csail.mit.edu/

Cite:

Eye Tracking for Everyone
K.Krafka*, A. Khosla*, P. Kellnhofer, H. Kannan, S. Bhandarkar, W. Matusik and A. Torralba
IEEE Conference on Computer Vision and Pattern Recognition (CVPR), 2016

@inproceedings{cvpr2016_gazecapture,
Author = {Kyle Krafka and Aditya Khosla and Petr Kellnhofer and Harini Kannan and Suchendra Bhandarkar and Wojciech Matusik and Antonio Torralba},
Title = {Eye Tracking for Everyone},
Year = {2016},
Booktitle = {IEEE Conference on Computer Vision and Pattern Recognition (CVPR)}
}

'''

def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

# Change there flags to control what happens.
doLoad = True # Load checkpoint at the beginning
doTest = True
data_path = 'TabletGazeDataset'

device = 'cuda'
cpu_workers = 2
cuda_workers = 8
epochs = 25
batch_size = 64 if device == 'cpu' else torch.cuda.device_count() * 50

base_lr = 0.0001
momentum = 0.9
weight_decay = 1e-4
print_freq = 10
prec1 = 0
best_prec1 = 1e20
lr = base_lr

count_test = 0
count = 0


def main():
    global args, best_prec1, weight_decay, momentum

    model = ITrackerModel()
    model = torch.nn.DataParallel(model)
    model.to(torch.device(device))
    imSize=(224,224)
    cudnn.benchmark = True
    cudnn.deterministic = False   

    epoch = 0
    if doLoad:
        saved = load_checkpoint()
        if saved:
            print('Loading checkpoint for epoch %05d with loss %.5f (which is the mean squared error not the actual linear error)...' % (saved['epoch'], saved['best_prec1']))
            state = saved['state_dict']
            try:
                model.module.load_state_dict(state)
            except:
                model.load_state_dict(state)
            epoch = saved['epoch']
            best_prec1 = saved['best_prec1']
        else:
            print('Warning: Could not read checkpoint!')

    dataTrain = TabletGazePostprocessData(dataPath = data_path, split='train', imSize = imSize)
    dataVal = TabletGazePostprocessData(dataPath = data_path, split='test', imSize = imSize)
   
    train_loader = torch.utils.data.DataLoader(
        dataTrain,
        batch_size=batch_size, shuffle=True,
        num_workers=cpu_workers if device == 'cpu' else cuda_workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        dataVal,
        batch_size=batch_size, shuffle=False,
        num_workers=cpu_workers if device == 'cpu' else cuda_workers, pin_memory=True)

    criterion = nn.MSELoss().to(torch.device(device)) # Mean squared error loss function

    optimizer = torch.optim.SGD(model.parameters(), lr, # Uses stochastic gradient descent
                                momentum=momentum,
                                weight_decay=weight_decay)

    # Quick test
    if doTest:
        validate(val_loader, model, criterion, epoch)
        return

    for epoch in range(0, epoch):
        adjust_learning_rate(optimizer, epoch)
        
    for epoch in range(epoch, epochs):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion, epoch)

        # remember best prec@1 and save checkpoint
        is_best = prec1 < best_prec1
        best_prec1 = min(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best)


def train(train_loader, model, criterion,optimizer, epoch):
    global count
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    dev = torch.device(device)

    for i, (imFace, imEyeL, imEyeR, faceGrid, gaze) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        imFace = imFace.to(dev)
        imEyeL = imEyeL.to(dev)
        imEyeR = imEyeR.to(dev)
        faceGrid = faceGrid.to(dev)
        gaze = gaze.to(dev)

        # compute output
        output = model(imFace, imEyeL, imEyeR, faceGrid)

        loss = criterion(output, gaze)

        losses.update(loss.data.item(), imFace.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        count = count + 1

        print('Epoch (train): [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
            epoch, i, len(train_loader), batch_time=batch_time,
            data_time=data_time, loss=losses))


def validate(val_loader, model, criterion, epoch):
    global count_test
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    lossesLin = AverageMeter()

    # switch to evaluate mode
    model.eval()
    end = time.time()
    for i, (imFace, imEyeL, imEyeR, faceGrid, gaze) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        imFace = imFace.to(device=device)
        imEyeL = imEyeL.to(device=device)
        imEyeR = imEyeR.to(device=device)
        faceGrid = faceGrid.to(device=device)
        gaze = gaze.to(device=device)

        # compute output
        with torch.no_grad():
            output = model(imFace, imEyeL, imEyeR, faceGrid)

        loss = criterion(output, gaze)

        lossLin = output - gaze
        lossLin = torch.mul(lossLin, lossLin)
        lossLin = torch.sum(lossLin, 1)
        lossLin = torch.mean(torch.sqrt(lossLin))

        losses.update(loss.data.item(), imFace.size(0))
        lossesLin.update(lossLin.item(), imFace.size(0))

        # compute gradient and do SGD step
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        print('Epoch (val): [{0}][{1}/{2}]\t'
                'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                'Error L2 {lossLin.val:.4f} ({lossLin.avg:.4f})\t'.format(
            epoch, i, len(val_loader), batch_time=batch_time,
            loss=losses, lossLin=lossesLin))

    return lossesLin.avg

CHECKPOINTS_PATH = '.'


def load_checkpoint(filename='checkpoint.pth.tar'):
    filename = os.path.join(CHECKPOINTS_PATH, filename)
    print(filename)
    if not os.path.isfile(filename):
        return None
    state = torch.load(filename)
    return state


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    if not os.path.isdir(CHECKPOINTS_PATH):
        os.makedirs(CHECKPOINTS_PATH, 0o777)
    bestFilename = os.path.join(CHECKPOINTS_PATH, 'best_' + filename)
    filename = os.path.join(CHECKPOINTS_PATH, filename)
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, bestFilename)


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = base_lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.state_dict()['param_groups']:
        param_group['lr'] = lr


if __name__ == "__main__":
    main()
    print('DONE')
